In [1]:
import rasterio
from rasterio.windows import Window
from pathlib import Path
from pyproj import Transformer
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
from rasterio.crs import CRS


/mnt/trappist-r0/cmarshak/mambaforge/envs/dist-s1-val/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
confirmed_products_dir = Path('confirmed_products')

In [59]:
MGRS_TILE_ID = '50NPL'
MGRS_TILE_ID

'50NPL'

In [49]:
import rasterio
from rasterio.windows import Window
from pyproj import Transformer

def get_pixel_value(tif, lon, lat):
    with rasterio.open(tif) as src:
        transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
        
        x_utm, y_utm = transformer.transform(lon, lat)
        
        row, col = src.index(x_utm, y_utm)
        if row < 0 or row >= src.height or col < 0 or col >= src.width:
            raise ValueError("Point is outside the bounds of the raster")
        
        # Create window and read pixel value
        window = Window(col, row, 1, 1)
        val = src.read(1, window=window)[0, 0]
    
    return val

In [50]:
mgrs_ts_dir = confirmed_products_dir / MGRS_TILE_ID
mgrs_ts_prods = sorted(list(mgrs_ts_dir.glob('OPERA*/')))
mgrs_ts_prods[:2]


[PosixPath('confirmed_products/33QTU/OPERA_L3_DIST-ALERT-S1_T33QTU_20240116T171541Z_20251001T025943Z_S1_30_v0.1'),
 PosixPath('confirmed_products/33QTU/OPERA_L3_DIST-ALERT-S1_T33QTU_20240121T172349Z_20251001T093945Z_S1_30_v0.1')]

In [51]:
mgrs_ts_status = [prod_dir / f'{prod_dir.name}_GEN-DIST-STATUS.tif' for prod_dir in mgrs_ts_prods]
mgrs_ts_status[:2]

[PosixPath('confirmed_products/33QTU/OPERA_L3_DIST-ALERT-S1_T33QTU_20240116T171541Z_20251001T025943Z_S1_30_v0.1/OPERA_L3_DIST-ALERT-S1_T33QTU_20240116T171541Z_20251001T025943Z_S1_30_v0.1_GEN-DIST-STATUS.tif'),
 PosixPath('confirmed_products/33QTU/OPERA_L3_DIST-ALERT-S1_T33QTU_20240121T172349Z_20251001T093945Z_S1_30_v0.1/OPERA_L3_DIST-ALERT-S1_T33QTU_20240121T172349Z_20251001T093945Z_S1_30_v0.1_GEN-DIST-STATUS.tif')]

In [52]:
mgrs_ts_dates = [pd.to_datetime(prod_dir.name.split('_')[4]).date() for prod_dir in mgrs_ts_prods]
mgrs_ts_dates[:2]

[datetime.date(2024, 1, 16), datetime.date(2024, 1, 21)]

# Dist Tables

In [53]:
df_val = pd.read_csv('../tables/reference_data/selectedpointsLL.csv')
df_val.head()

,ID,Block,subID,blockStratum,substratum,zone,x,y,centx,centy,long,lat,MGRS
0,30961_1,30961,1,treelossTF,3,33,372932.570128,545394.967628,372945,545385,13.854048,4.933158,33NUF
1,30961_2,30961,2,treelossTF,2,33,372728.616472,542585.990364,372735,542595,13.852197,4.907920,33NUF
2,30961_3,30961,3,treelossTF,1,33,373846.916077,547093.248106,373845,547095,13.862138,4.948639,33NUF
3,30961_4,30961,4,treelossTF,4,33,371390.914129,546069.082218,371385,546075,13.839969,4.939375,33NUF
4,30961_5,30961,5,treelossTF,3,33,374504.487544,546457.554880,374505,546465,13.868099,4.942951,33NUF


In [54]:
df_val_mgrs = df_val[df_val['MGRS'] == MGRS_TILE_ID].reset_index(drop=True)
sites_in_mgrs_tile = df_val_mgrs.to_dict(orient='records')
sites_in_mgrs_tile[:2]

[{'ID': '116332_1',
  'Block': 116332,
  'subID': 1,
  'blockStratum': 'gen',
  'substratum': 2,
  'zone': 33,
  'x': 242657.835980735,
  'y': 1796125.72502618,
  'centx': 242655,
  'centy': 1796115,
  'long': 12.5925201960045,
  'lat': 16.2320438984228,
  'MGRS': '33QTU'},
 {'ID': '116332_2',
  'Block': 116332,
  'subID': 2,
  'blockStratum': 'gen',
  'substratum': 2,
  'zone': 33,
  'x': 245545.573985477,
  'y': 1796593.38461452,
  'centx': 245535,
  'centy': 1796595,
  'long': 12.6193937318321,
  'lat': 16.2366834399998,
  'MGRS': '33QTU'}]

In [55]:
tables = []
for site in tqdm(sites_in_mgrs_tile[1:], desc='Processing sites'):
    site_id = site['ID']
    site_lon = site['long']
    site_lat = site['lat'] 

    labels = [get_pixel_value(tif, site_lon, site_lat) for tif in mgrs_ts_status]

    df_dist_s1_table_site = pd.DataFrame({
        'site_id': [site_id] * len(labels),
        'date': mgrs_ts_dates,
        # 'longitude': [site_lon] * len(labels),
        # 'latitude': [site_lat] * len(labels),
        'labels': labels,
    })
    tables.append(df_dist_s1_table_site)

tables[0].head()


Processing sites: 100%|█| 9/9 [00:01<00:00,  8.6


,site_id,date,labels
0,116332_2,2024-01-16,0
1,116332_2,2024-01-21,0
2,116332_2,2024-01-28,0
3,116332_2,2024-02-02,0
4,116332_2,2024-02-09,0


In [56]:
for table in tables:
    site_id = table['site_id'].tolist()[0]
    block_id = site_id.split('_')[0]
    table_dir = Path(f'dist_s1_label_tables/{block_id}')
    table_dir.mkdir(exist_ok=True, parents=True)
    table_path = table_dir / f'{site_id}.csv'
    table.to_csv(table_path, index=False)


# Generate Site Vector Table

In [57]:
df_site_geo = gpd.GeoDataFrame(df_val_mgrs[['ID']],
                               geometry=gpd.points_from_xy(df_val_mgrs.long, 
                                                           df_val_mgrs.lat), 
                               crs=CRS.from_epsg(4326))
df_site_geo.head()


,ID,geometry
0,116332_1,POINT (12.59252 16.23204)
1,116332_2,POINT (12.61939 16.23668)
2,116332_3,POINT (12.59444 16.21255)
3,116332_4,POINT (12.6247 16.26195)
4,116332_5,POINT (12.60629 16.27664)


In [58]:
df_site_geo.to_file(table_dir / 'sites.geojson', driver='GeoJSON')